# Table of Contents



**1. Installation Instructions**

  **2.   Data Preprocessing**

**3. Model Building**

# Installation Instructions 


1.   **Clone** our GitHub repository


In [ ]:
! git clone https://github.com/s-azaar/document_layout_segmentation.git      

2.   **Download** the requirements of yolov3



In [ ]:
! cd document_layout_segmentation/yolov3/installation && cat requirements_colab.txt | xargs -n 1 -L 1 pip install
! pip install unrar

3.  **Download** Plates' dataset


In [ ]:
! wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=16uHETYgYABxAbcpsTz2m6zgWxMdVJuF6' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=16uHETYgYABxAbcpsTz2m6zgWxMdVJuF6" -O PalestinePlateDataSet.rar && rm -rf /tmp/cookies.txt


# Data Preprocessing

1. **Unrar** the cars' dataset.

In [ ]:
! unrar x PalestinePlateDataSet.rar


2.   **Rmoave** the first line which is the plate number



In [5]:
# Import the necessary libraries 
import glob
import os.path
from tqdm.notebook import tqdm

In [6]:
# Added files directory
dataset_dir = "PalestinePlateDataSet/";
anno_dir = "annotation/";
images_dir = "images/";

In [ ]:
# Run removing process
for name in tqdm(glob.glob(dataset_dir+anno_dir+'*.txt')):
    with open(name, 'r') as fin:
      data = fin.read().splitlines(True)
    with open(name, 'w') as fout:
      fout.writelines(data[1:])


3. **ADD** Plate word at the beginning of each file in the annotation files for the training purpose 

In [ ]:
# Run removing process
for name in tqdm(glob.glob(dataset_dir+anno_dir+'*.txt')):
    with open(name, 'r') as fin:
      data = fin.read()
    with open(name, 'w') as fout:
      fout.writelines("0"+" "+data)
    

4. **Remove** unneeded  images because of their quality 

In [ ]:
# Run removing process

with open(dataset_dir+"skipped_images.txt", 'r') as fin:
    data = fin.read().splitlines(False)

for name in data:
  try:
      os.remove(dataset_dir+anno_dir+name+".txt")
  except:
      print(name)
  os.remove(dataset_dir+images_dir+name+".jpg")

# to remove the images that don't't exist
for name_image in tqdm(glob.glob(dataset_dir+images_dir+'*.jpg')):
    name_anno = name_image.split("/")[2].split(".")[0]+".txt"
    if  os.path.exists(dataset_dir+anno_dir+name_anno):
        continue
    else:
       os.remove(name_image)

5. **Adjust** the bounded boxes 

In [17]:
# Adjust the bounding box
from PIL import Image 

for name in tqdm(glob.glob(dataset_dir+anno_dir+'*.txt')):
    with open(name, 'r') as fin:
      data = fin.read()
    
    img_file = name.split("/")
    img = Image.open(dataset_dir  + images_dir  + img_file[2].split(".")[0] + ".jpg");
    width, height = img.size
    remove_space = data.split(" ")
    img_file = name;
    label = remove_space[0];
    x1 =  int (remove_space[1]);
    y1 =  int (remove_space[2]);
    x2 =  int (remove_space[3]);
    y2 =  int (remove_space[4]);

    x_c = str(((x1 + x2)/2)/width);
    y_c = str(((y1 + y2)/2)/height);
    w = str((x2 - x1)/width);
    h = str((y2 - y1)/height);

    with open(name, 'w') as filetowrite:
          filetowrite.writelines(label + " " + x_c + " " + y_c + " " + w + " " + h)
    


6. **ADD** classs file

In [7]:
! touch classes.txt
! echo Plate > classes.txt

# Model Building 

1. **Import** all necessary classes

In [5]:
import os
import sys
sys.path.append("document_layout_segmentation/yolov3/lib")
from train_detector import Detector
detect_image = Detector();

2. **Load** all necessary directory

In [6]:
#dataset directories
img_dir = "PalestinePlateDataSet/images/"
label_dir = "PalestinePlateDataSet/annotation/"
class_list_file = "classes.txt"

3. **Start** learning process

In [ ]:
detect_image.set_model(model_name="yolov3");
detect_image.set_train_dataset(img_dir, label_dir, class_list_file, batch_size=16)
detect_image.set_val_dataset(img_dir, label_dir)
detect_image.set_hyperparams(optimizer="sgd", lr=0.003, multi_scale=False, evolve=False);
detect_image.Train(num_epochs=30);